# Unsupervisioned Learning (Aprendizado Não Supervisionado)

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo
* Introdução
* Cross-Validation (Validação Cruzada)
  * The Validation Set Approach (A Abordagem do Conjunto de Validação)
  * Leave-One-Out Cross-Validation (Validação Cruzada Leave-One-Out)
  * k-Fold Cross-Validation (Validação Cruzada k-Fold)
* The Bootstrap

### Referências

* [An Introduction to Statistical Learning](https://www.statlearning.com/) (ISL) by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani
  * Capítulo 5***
* [The Elements of Statistical Learning](https://hastie.su.domains/ElemStatLearn/) (ESL) by Trevor Hastie, Robert Tibshirani and Jerome Friedman : 
  * Capítulo 7

***Disclaimer:*** *O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer feedback ou sugestão de melhoria.*

In [2]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# General packages
import pandas as pd
import numpy as np
import seaborn as sns
import time
from scipy.stats import multivariate_normal

# Sklean
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree

# Import matplotlib for graphs
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import axes3d
from IPython.display import clear_output

# Set global parameters
plt.style.use('seaborn-v0_8-white')
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['figure.titlesize'] = 20
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

# Figure 10.1 a
def make_figure_10_1a(df_dim2, df_weights):

    # Init
    fig, ax1 = plt.subplots(figsize=(8,8))
    ax1.set_title('Figure 10.1');

    # Plot Principal Components 1 and 2
    for i in df_dim2.index:
        ax1.annotate(i, (df_dim2.PC1.loc[i], -df_dim2.PC2.loc[i]), ha='center', fontsize=14)

    # Plot reference lines
    m = np.max(np.abs(df_dim2.values))*1.2
    ax1.hlines(0,-m,m, linestyles='dotted', colors='grey')
    ax1.vlines(0,-m,m, linestyles='dotted', colors='grey')
    ax1.set_xlabel('First Principal Component')
    ax1.set_ylabel('Second Principal Component')
    ax1.set_xlim(-m,m); ax1.set_ylim(-m,m)

    # Plot Principal Component loading vectors, using a second y-axis.
    ax1b = ax1.twinx().twiny() 
    ax1b.set_ylim(-1,1); ax1b.set_xlim(-1,1)
    for i in df_weights[['PC1', 'PC2']].index:
        ax1b.annotate(i, (df_weights.PC1.loc[i]*1.05, -df_weights.PC2.loc[i]*1.05), color='orange', fontsize=16)
        ax1b.arrow(0,0,df_weights.PC1[i], -df_weights.PC2[i], color='orange', lw=2)
        
        
# Figure 10.1 b
def make_figure_10_1b(df_dim2, df_dim2_u, df_weights, df_weights_u):

    # Init
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(18,9))

    # Scaled PCA
    for i in df_dim2.index:
        ax1.annotate(i, (df_dim2.PC1.loc[i], -df_dim2.PC2.loc[i]), ha='center', fontsize=14)
    ax1b = ax1.twinx().twiny() 
    ax1b.set_ylim(-1,1); ax1b.set_xlim(-1,1)
    for i in df_weights[['PC1', 'PC2']].index:
        ax1b.annotate(i, (df_weights.PC1.loc[i]*1.05, -df_weights.PC2.loc[i]*1.05), color='orange', fontsize=16)
        ax1b.arrow(0,0,df_weights.PC1[i], -df_weights.PC2[i], color='orange', lw=2)
    ax1.set_title('Scaled')

    # Unscaled PCA
    for i in df_dim2_u.index:
        ax2.annotate(i, (df_dim2_u.PC1.loc[i], -df_dim2_u.PC2.loc[i]), ha='center', fontsize=14)
    ax2b = ax2.twinx().twiny() 
    ax2b.set_ylim(-1,1); ax2b.set_xlim(-1,1)
    for i in df_weights_u[['PC1', 'PC2']].index:
        ax2b.annotate(i, (df_weights_u.PC1.loc[i]*1.05, -df_weights_u.PC2.loc[i]*1.05), color='orange', fontsize=16)
        ax2b.arrow(0,0,df_weights_u.PC1[i], -df_weights_u.PC2[i], color='orange', lw=2)
    ax2.set_title('Unscaled')

    # Plot reference lines
    for ax,df in zip((ax1,ax2), (df_dim2,df_dim2_u)):
        m = np.max(np.abs(df.values))*1.2
        ax.hlines(0,-m,m, linestyles='dotted', colors='grey')
        ax.vlines(0,-m,m, linestyles='dotted', colors='grey')
        ax.set_xlabel('First Principal Component')
        ax.set_ylabel('Second Principal Component')
        ax.set_xlim(-m,m); ax.set_ylim(-m,m)


# Figure 10.2
def make_figure_10_2(pca4):

    # Init
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))
    fig.suptitle('Figure 10.2');

    # Relative 
    ax1.plot([1,2,3,4], pca4.explained_variance_ratio_)
    ax1.set_ylabel('Prop. Variance Explained')
    ax1.set_xlabel('Principal Component');

    # Cumulative
    ax2.plot([1,2,3,4], np.cumsum(pca4.explained_variance_ratio_))
    ax2.set_ylabel('Cumulative Variance Explained');
    ax2.set_xlabel('Principal Component');


# Figure new 1
def make_new_figure_1(X):
    
    # Init
    fig, ax = plt.subplots(figsize=(6, 5))
    fig.suptitle("Baseline")

    # Plot
    ax.scatter(X[:,0], X[:,1], s=50, alpha=0.5, c='k') 
    ax.set_xlabel('X0'); ax.set_ylabel('X1');
    

# Figure new 2
def make_new_figure_2(X, clusters0):
    
    # Init
    fig, ax = plt.subplots(figsize=(6, 5))
    fig.suptitle("Random assignment")

    # Plot
    ax.scatter(X[clusters0==0,0], X[clusters0==0,1], s=50, alpha=0.5) 
    ax.scatter(X[clusters0==1,0], X[clusters0==1,1], s=50, alpha=0.5)
    ax.set_xlabel('X0'); ax.set_ylabel('X1');


# Plot assignment
def plot_assignment(X, centroids, clusters, d, i):
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(6, 5))
    fig.suptitle("Iteration %.0f: inertia=%.1f" % (i,d))

    # Plot
    ax.clear()
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'];
    K = np.size(centroids,0)
    for k in range(K):
        ax.scatter(X[clusters==k,0], X[clusters==k,1], s=50, c=colors[k], alpha=0.5) 
        ax.scatter(centroids[k,0], centroids[k,1], marker = '*', s=300, color=colors[k])
        ax.set_xlabel('X0'); ax.set_ylabel('X1');
    
    # Show
    plt.show();
    
# Figure new 3
def make_new_figure_3(d):
    
    # Init
    plt.figure(figsize=(25, 10))
    plt.title('Hierarchical Clustering Dendrogram')

    # calculate full dendrogram
    plt.xlabel('sample index')
    plt.ylabel('distance')
    d
    plt.show()
    
    
# Figure new 4
def make_new_figure_4(linkages, titles):
    
    # Init
    fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(15,6))

    # Plot
    for linkage, t, ax in zip(linkages, titles, (ax1,ax2,ax3)):
        dendrogram(linkage, ax=ax)
        ax.set_title(t)
        
        
def get_cov_ellipse(distr, nstd, **kwargs):
    """
    Return a matplotlib Ellipse patch representing a standard distribution around the mean
    """

    # Find and sort eigenvalues and eigenvectors into descending order
    eigvals, eigvecs = np.linalg.eigh(distr.cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]

    # The anti-clockwise angle to rotate our ellipse by 
    vx, vy = eigvecs[:,0][0], eigvecs[:,0][1]
    theta = np.arctan2(vy, vx)

    # Width and height of ellipse to draw
    width, height = 2 * nstd * np.sqrt(eigvals)
    return Ellipse(xy=distr.mean, width=width, height=height,
                   angle=np.degrees(theta), **kwargs)


# Plot assignment
def plot_assignment_gmm(X, clusters, distr, i, logL):
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(6, 5))
    fig.suptitle(f"Iteration {i}, logL={logL:.2}")

    # Plot
    ax.clear()
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'];
    K = len(distr)
    for k in range(K):
        ax.scatter(X[clusters==k,0], X[clusters==k,1], s=50, c=colors[k], alpha=0.5) 
        ax.scatter(distr[k].mean[0], distr[k].mean[1], marker = '*', s=300, color=colors[k])
        for i in [0.5, 1, 2]:
            ax.add_artist(get_cov_ellipse(distr[k], nstd=i, color=colors[k], alpha=0.05))
        ax.set_xlabel('X0'); ax.set_ylabel('X1');
    
    # Show
    plt.show();

### Aprendizagem supervisionada vs. não supervisionada

A diferença entre aprendizagem supervisionada e aprendizagem não supervisionada é que no primeiro caso temos uma variável
que queremos prever, dado um conjunto de variáveis
.

Na aprendizagem não supervisionada não estamos interessados ​​em previsão, porque não temos uma variável de resposta associada
. Em vez disso, o objetivo é descobrir propriedades interessantes sobre as medições em
.

As questões que normalmente nos interessam são:

Agrupamento
Redução de dimensionalidade
Em geral, o aprendizado não supervisionado pode ser visto como uma extensão da análise exploratória de dados.

Redução da dimensionalidade
Trabalhar em espaços de alta dimensão pode ser indesejável por vários motivos; dados brutos geralmente são esparsos como consequência da maldição da dimensionalidade, e analisar os dados geralmente é computacionalmente intratável (difícil de controlar ou lidar).

A redução da dimensionalidade também pode ser útil para plotar dados de alta dimensionalidade.

Agrupamento
Agrupamento refere-se a um conjunto muito amplo de técnicas para encontrar subgrupos, ou clusters, em um conjunto de dados. Quando agrupamos as observações de um conjunto de dados, buscamos particioná-las em grupos distintos, de modo que as observações dentro de cada grupo sejam bastante semelhantes entre si, enquanto as observações em grupos diferentes sejam bastante diferentes entre si.

Nesta seção, nos concentramos nos seguintes algoritmos:

Agrupamento de K-means
Agrupamento hierárquico
Modelos de Mistura Gaussiana

Análise de Componentes Principais

Suponha que desejamos visualizar
observações com medições em um conjunto de
características,
, como parte de uma análise exploratória de dados.

Poderíamos fazer isso examinando gráficos de dispersão bidimensionais dos dados, cada um contendo as medições das n observações em duas das características. No entanto, existem
tais gráficos de dispersão; por exemplo, com
há
tramas!

A ACP fornece uma ferramenta para fazer exatamente isso. Ela encontra uma representação de baixa dimensão de um conjunto de dados que contém o máximo possível da variação.

Primeiro Componente Principal

O primeiro componente principal de um conjunto de recursos
é a combinação linear normalizada das características


que tem a maior variância.

Por normalizado, queremos dizer que
.

Computação PCA
Em outras palavras, o primeiro vetor de carga do componente principal resolve o problema de otimização

 
 
 
 
 

O objetivo que estamos maximizando é apenas a variância da amostra do
valores de
.

Após o primeiro componente principal
das características foi determinado, podemos encontrar o segundo componente principal
. O segundo componente principal é a combinação linear de
que tem variância máxima de todas as combinações lineares que não são correlacionadas com
.

Exemplo

Ilustramos o uso de PCA no USArrestsconjunto de dados.

Para cada um dos 50 estados dos Estados Unidos, o conjunto de dados contém o número de prisões por
residentes para cada um dos três crimes: Assault, Murder, e Rape.Também registramos a porcentagem da população em cada estado que vive em áreas urbanas, UrbanPop.

In [ ]:
# Load crime data
df = pd.read_csv('data/USArrests.csv', index_col=0)
df.head()

Dimensionamento de dados

Para tornar todas as características comparáveis, primeiro precisamos escaloná-las. Nesse caso, usamos a sklearn.preprocessing.scale()função para normalizar cada variável para que tenha média zero e variância unitária.

In [ ]:
# Scale data
X_scaled = pd.DataFrame(scale(df), index=df.index, columns=df.columns).values

eremos mais tarde quais são as implicações práticas da (não) escala.

Montagem
Vamos ajustar o PCA com 2 componentes.

In [ ]:
# Fit PCA with 2 components
pca2 = PCA(n_components=2).fit(X_scaled)

In [ ]:
# Get weights
weights = pca2.components_.T
df_weights = pd.DataFrame(weights, index=df.columns, columns=['PC1', 'PC2'])
df_weights

Projetando os dados
Como são os dados transformados?

In [ ]:
# Transform X to get the principal components
X_dim2 = pca2.transform(X_scaled)
df_dim2 = pd.DataFrame(X_dim2, columns=['PC1', 'PC2'], index=df.index)
df_dim2.head()

Visualização

A vantagem do PCA é que ele nos permite ver a variação em dimensões menores.

In [ ]:
make_figure_10_1a(df_dim2, df_weights)

PCA e Análise Espectral
Caso você não tenha notado, calcular os componentes principais é equivalente a calcular os autovetores da matriz de projeto
, ou seja, a matriz de variância-covariância de
. Na verdade, o que realizamos acima é uma decomposição da variância de
em componentes ortogonais.

O problema de maximização restrita acima pode ser reescrito em notação matricial como


Que tem a seguinte representação dupla


Se tomarmos as condições de primeira ordem

 
 
 

Definindo as derivadas como zero no ótimo, obtemos

 

Por isso,
é um autovetor da matriz de covariância
, e o vetor maximizador será aquele associado ao maior autovalor 
.

Autovalores e autovetores
Agora podemos verificar novamente usando numpyo pacote de álgebra linear.

In [ ]:
eigenval, eigenvec = np.linalg.eig(X_scaled.T @ X_scaled)
data = np.concatenate((eigenvec,eigenval.reshape(1,-1)))
idx = list(df.columns) + ['Eigenvalue']
df_eigen = pd.DataFrame(data, index=idx, columns=['PC1', 'PC2','PC3','PC4'])

df_eigen

A decomposição espectral da variância de
gera um conjunto de vetores ortogonais (autovetores) com diferentes magnitudes (autovalores). Os autovalores nos indicam a quantidade de variância dos dados naquela direção.

Se combinarmos os autovetores, formamos uma matriz de projeção
que podemos usar para transformar as variáveis ​​originais:


In [ ]:
X_transformed = X_scaled @ eigenvec
df_transformed = pd.DataFrame(X_transformed, index=df.index, columns=['PC1', 'PC2','PC3','PC4'])

df_transformed.head()

Este é exatamente o conjunto de dados que obtivemos antes.

### Escalando as Variáveis

Os resultados obtidos ao realizarmos a ACP também dependerão de as variáveis ​​terem sido escalonadas individualmente. De fato, a variância de uma variável depende de sua magnitude.

In [ ]:
# Variables variance
df.var(axis=0)

Consequentemente, se executarmos a ACP nas variáveis ​​não escalonadas, o primeiro vetor de carga do componente principal terá uma carga muito grande para Assault, já que essa variável tem de longe a maior variância.

In [ ]:
# Fit PCA with unscaled varaibles
X = df.values
pca2_u = PCA(n_components=2).fit(X)

In [ ]:
# Get weights
weights_u = pca2_u.components_.T
df_weights_u = pd.DataFrame(weights_u, index=df.columns, columns=['PC1', 'PC2'])
df_weights_u

In [ ]:
# Transform X to get the principal components
X_dim2_u = pca2_u.transform(X)
df_dim2_u = pd.DataFrame(X_dim2_u, columns=['PC1', 'PC2'], index=df.index)

Plotagem
Podemos comparar as representações dimensionais inferiores com e sem escala.

In [ ]:
make_figure_10_1b(df_dim2, df_dim2_u, df_weights, df_weights_u)

Conforme previsto, o primeiro vetor de carga de componentes principais coloca quase todo o seu peso em Assault, enquanto o segundo vetor de carga de componentes principais coloca quase todo o seu peso em UrpanPop. Comparando isso com o gráfico da esquerda, vemos que a escala de fato tem um efeito substancial nos resultados obtidos. No entanto, esse resultado é simplesmente uma consequência das escalas em que as variáveis ​​foram medidas.

A Proporção de Variância Explicada
Podemos agora fazer uma pergunta natural: quanta informação em um determinado conjunto de dados é perdida ao projetar as observações nos primeiros componentes principais? Ou seja, quanta da variância nos dados não está contida nos primeiros componentes principais? De forma mais geral, estamos interessados ​​em saber a proporção da variância explicada (PVE) por cada componente principal.

In [ ]:
# Four components
pca4 = PCA(n_components=4).fit(X_scaled)

In [ ]:
# Variance of the four principal components
pca4.explained_variance_

Interpretação
Podemos calculá-lo em porcentagem da variância total.

In [ ]:
# As a percentage of the total variance
pca4.explained_variance_ratio_

No Arrestconjunto de dados, o primeiro componente principal explica
da variância nos dados, e o próximo componente principal explica
da variância. Juntos, os dois primeiros componentes principais explicam quase
da variância dos dados, e os dois últimos componentes principais explicam apenas
da variância.

Plotagem
Podemos traçar em um gráfico a porcentagem da variância explicada, em relação ao número de componentes.

In [ ]:
make_figure_10_2(pca4)

Quantos componentes principais?


Em geral, um
matriz de dados
tem
componentes principais distintos. No entanto, normalmente não nos interessamos por todos eles; em vez disso, gostaríamos de usar apenas os primeiros componentes principais para visualizar ou interpretar os dados.

Normalmente, decidimos o número de componentes principais necessários para visualizar os dados examinando um gráfico de declive .

Entretanto, não há uma maneira objetiva bem aceita de decidir quantos componentes principais são suficientes.

Agrupamento de K-Means
A ideia por trás do agrupamento K-means é que um bom agrupamento é aquele em que a variação dentro do agrupamento é a menor possível. Portanto, queremos resolver o problema

 
 

onde
é um cluster e
é uma medida da quantidade pela qual as observações dentro de um cluster diferem umas das outras.

Existem muitas maneiras possíveis de definir este conceito, mas de longe a escolha mais comum envolve a distância euclidiana ao quadrado . Ou seja, definimos

 
 
 

onde
denota o número de observações no
conjunto.

Algoritmo
Atribuir um número aleatoriamente, de
para
, para cada uma das observações. Elas servem como atribuições iniciais de cluster para as observações.

Repita até que as atribuições do cluster parem de mudar:

a) Para cada um dos
clusters, calcule o centróide do cluster. O centróide do cluster k é o vetor do
recursos significam para as observações no
conjunto.

b) Atribua cada observação ao cluster cujo centroide é o mais próximo (onde o mais próximo é definido usando a distância euclidiana).

Gerar os dados
Primeiro, geramos um conjunto de dados bidimensional.

In [ ]:
# Simulate data
np.random.seed(123)
X = np.random.randn(50,2)
X[0:25, 0] = X[0:25, 0] + 3
X[0:25, 1] = X[0:25, 1] - 4

In [ ]:
make_new_figure_1(X)

Etapa 1: atribuição aleatória
Agora vamos atribuir os dados aleatoriamente a dois clusters.

In [ ]:
# Init clusters
K = 2
clusters0 = np.random.randint(K,size=(np.size(X,0)))

make_new_figure_2(X, clusters0)

Etapa 2: estimar distribuições
O que são os novos centróides?

In [ ]:
# Compute new centroids
def compute_new_centroids(X, clusters):
    K = len(np.unique(clusters))
    centroids = np.zeros((K,np.size(X,1)))
    for k in range(K):
        if sum(clusters==k)>0:
            centroids[k,:] = np.mean(X[clusters==k,:], axis=0)
        else:
            centroids[k,:] = np.mean(X, axis=0)
    return centroids

In [ ]:
# Print
centroids0 = compute_new_centroids(X, clusters0)
print(centroids0)

Traçando os centróides
Vamos adicionar os centroides ao gráfico.

In [ ]:
# Plot
plot_assignment(X, centroids0, clusters0, 0, 0)

Etapa 3: atribuir dados aos clusters
Agora podemos atribuir os dados aos clusters, de acordo com o centróide mais próximo.

In [ ]:
# Assign X to clusters
def assign_to_cluster(X, centroids):
    K = np.size(centroids,0)
    dist = np.zeros((np.size(X,0),K))
    for k in range(K):
        dist[:,k] = np.mean((X - centroids[k,:])**2, axis=1)
    clusters = np.argmin(dist, axis=1)
    
    # Compute inertia
    inertia = 0
    for k in range(K):
        if sum(clusters==k)>0:
            inertia += np.sum((X[clusters==k,:] - centroids[k,:])**2)
    return clusters, inertia

Plotando dados atribuídos

In [ ]:
# Get cluster assignment
[clusters1,d] = assign_to_cluster(X, centroids0)

In [ ]:
# Plot
plot_assignment(X, centroids0, clusters1, d, 1)

Algoritmo completo
Agora temos todos os componentes para prosseguir iterativamente.

In [ ]:
def kmeans_manual(X, K):

    # Init
    i = 0
    d0 = 1e4
    d1 = 1e5
    clusters = np.random.randint(K,size=(np.size(X,0)))

    # Iterate until convergence
    while np.abs(d0-d1) > 1e-10:
        d1 = d0
        centroids = compute_new_centroids(X, clusters)
        [clusters, d0] = assign_to_cluster(X, centroids)
        plot_assignment(X, centroids, clusters, d0, i)
        i+=1

Plotando o agrupamento k-means

In [ ]:
# Test
kmeans_manual(X, K)

Aqui, as observações podem ser facilmente plotadas porque são bidimensionais. Se houvesse mais de duas variáveis, poderíamos realizar a ACP e plotar os dois primeiros vetores de pontuação dos componentes principais.

Mais clusters
No exemplo anterior, sabíamos que realmente havia dois clusters porque geramos os dados. No entanto, para dados reais, em geral, não sabemos o número real de clusters. Poderíamos ter realizado o agrupamento K-means neste exemplo com K = 3. Se fizermos isso, o agrupamento K-means dividirá os dois clusters "reais", já que não possui informações sobre eles:

In [ ]:
# K=3
kmeans_manual(X, 3)

Pacote Sklearn
A função automatizada em sklearnpersorm
- significa que o agrupamento é KMeans.

In [ ]:
# SKlearn algorithm
km1 = KMeans(n_clusters=3, n_init=1, random_state=1)
km1.fit(X)

In [ ]:
KMeans(n_clusters=3, n_init=1, random_state=1)

Plotagem
Podemos plotar a atribuição gerada pela KMeansfunção.

In [ ]:
# Plot
plot_assignment(X, km1.cluster_centers_, km1.labels_, km1.inertia_, km1.n_iter_)

Como podemos ver, os resultados são diferentes nos dois algoritmos? Por quê?
-means é suscetível aos valores iniciais. Uma maneira de resolver esse problema é executar o algoritmo várias vezes e reportar apenas os melhores resultados.

Atribuição Inicial
Para executar a Kmeans()função em Python com múltiplas atribuições iniciais de cluster, usamos o n_initargumento (padrão: 10). Se um valor n_initmaior que um for usado, o agrupamento K-means será realizado usando múltiplas atribuições aleatórias, e a Kmeans()função reportará apenas os melhores resultados.

In [ ]:
# 30 runs
km_30run = KMeans(n_clusters=3, n_init=30, random_state=1).fit(X)
plot_assignment(X, km_30run.cluster_centers_, km_30run.labels_, km_30run.inertia_, km_30run.n_iter_)

Melhores Práticas
Geralmente, é recomendável sempre executar o agrupamento K-means com um valor grande de n_init, como 20 ou 50, para evitar ficar preso em um ótimo local indesejado.

Ao realizar o agrupamento K-means, além de usar múltiplas atribuições iniciais de clusters, também é importante definir uma semente aleatória usando o random_stateparâmetro. Dessa forma, as atribuições iniciais de clusters podem ser replicadas e a saída do K-means será totalmente reproduzível.

Agrupamento hierárquico
Uma desvantagem potencial do agrupamento K-means é que ele exige que pré-especificemos o número de agrupamentos
.

O agrupamento hierárquico é uma abordagem alternativa que não exige que nos comprometamos com uma escolha específica de
.

O Dendograma
O agrupamento hierárquico tem uma vantagem adicional sobre o agrupamento K-means, pois resulta em uma representação atraente baseada em árvore das observações, chamada dendrograma .

In [ ]:
d = dendrogram(
        linkage(X, "complete"),
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
    )

Interpretação
Cada folha do dendrograma representa uma observação.

À medida que subimos na árvore, algumas folhas começam a se fundir em galhos. Isso corresponde a observações semelhantes entre si. À medida que subimos na árvore, os próprios galhos se fundem, seja com folhas ou com outros galhos. Quanto mais cedo (na parte inferior da árvore) as fusões ocorrerem, mais semelhantes serão os grupos de observações entre si.

Podemos usar o dendograma para entender o quão semelhantes são duas observações: podemos procurar o ponto na árvore onde os ramos que contêm essas duas observações se fundem pela primeira vez. A altura dessa fusão, medida no eixo vertical, indica o quão diferentes são as duas observações. Assim, observações que se fundem na base da árvore são bastante semelhantes entre si, enquanto observações que se fundem perto do topo da árvore tendem a ser bastante diferentes.

O termo hierárquico se refere ao fato de que os clusters obtidos pelo corte do dendrograma em uma determinada altura são necessariamente aninhados dentro dos clusters obtidos pelo corte do dendrograma em qualquer altura maior.

O Algoritmo de Agrupamento Hierárquico
Comece com
observações e uma medida (como a distância euclidiana) de todos os
Dissimilaridades em pares. Trate cada observação como um cluster separado.

Para

a) Examine todas as diferenças entre pares de grupos entre os
clusters e identifique o par de clusters que são menos dissimilares (ou seja, mais semelhantes). Funda esses dois clusters. A dissimilaridade entre esses dois clusters indica a altura no dendrograma em que a fusão deve ser colocada.

b) Calcule as novas diferenças entre pares de clusters entre os
clusters restantes.

A função de ligação
Temos um conceito de dissimilaridade entre pares de observações, mas como definimos a dissimilaridade entre dois clusters se um ou ambos os clusters contêm múltiplas observações?

O conceito de dissimilaridade entre um par de observações precisa ser estendido a um par de grupos de observações. Essa extensão é alcançada desenvolvendo a noção de ligação , que define a dissimilaridade entre dois grupos de observações.

Ligações
Os quatro tipos mais comuns de ligação são:

Completo : Dissimilaridade máxima entre grupos. Calcule todas as dissimilaridades em pares entre as observações no grupo A e as observações no grupo B e registre a maior dessas dissimilaridades.
Único : Dissimilaridade mínima entre grupos. Calcule todas as dissimilaridades em pares entre as observações no grupo A e as observações no grupo B e registre a menor dessas dissimilaridades.
Média : Dissimilaridade média entre grupos. Calcule todas as dissimilaridades em pares entre as observações no grupo A e as observações no grupo B e registre a média dessas dissimilaridades.
Centroide : Dissimilaridade entre o centroide do cluster A (um vetor médio de comprimento p) e o centroide do cluster B. A ligação do centroide pode resultar em inversões indesejáveis.
Ligações médias, completas e simples são as mais populares entre os estatísticos. Ligações médias e completas são geralmente preferidas à ligação simples, pois tendem a produzir dendrogramas mais equilibrados. A ligação centroide é frequentemente usada em genômica, mas sofre de uma grande desvantagem: pode ocorrer uma inversão, na qual dois clusters se fundem a uma altura abaixo de qualquer um dos clusters individuais no dendrograma. Isso pode levar a dificuldades na visualização, bem como na interpretação do dendrograma.

In [ ]:
# Init
linkages = [hierarchy.complete(X), hierarchy.average(X), hierarchy.single(X)]
titles = ['Complete Linkage', 'Average Linkage', 'Single Linkage']

Plotagem

In [ ]:
make_new_figure_4(linkages, titles)

Para esses dados, tanto a vinculação completa quanto a média geralmente separam as observações em seus grupos corretos.

Modelos de Mistura Gaussiana
Métodos de agrupamento, como agrupamento hierárquico e K-means, são baseados em heurísticas e dependem principalmente da descoberta de agrupamentos cujos membros são próximos uns dos outros, conforme medido diretamente com os dados (nenhum modelo de probabilidade envolvido).

Os Modelos de Mistura Gaussiana pressupõem que os dados foram gerados por múltiplas distribuições gaussianas multivariadas. O objetivo do algoritmo é recuperar essas distribuições latentes.

As vantagens em relação ao K-means são

uma interpretação estrutural dos parâmetros
gera automaticamente probabilidades de classe
pode gerar grupos de observações que não estão necessariamente próximos uns dos outros
Algoritmo
Atribuir um número aleatoriamente, de
para
, para cada uma das observações. Elas servem como atribuições iniciais de cluster para as observações.

Repita até que as atribuições do cluster parem de mudar:

a) Para cada um dos
clusters, calculamos sua média e variância. A principal diferença com o K-means é que também calculamos a matriz de variância.

b) Atribua cada observação ao seu cluster mais provável.

Conjunto de dados
Vamos usar os mesmos dados que usamos para k-means, para uma comparação direta.

In [ ]:
make_new_figure_1(X)

Etapa 1: atribuição aleatória
Vamos também usar a mesma atribuição aleatória do algoritmo K-means.

In [ ]:
make_new_figure_2(X, clusters0)

Etapa 2: calcular distribuições
Quais são as novas distribuições?

In [ ]:
# Compute new centroids
def compute_distributions(X, clusters):
    K = len(np.unique(clusters))
    distr = []
    for k in range(K):
        if sum(clusters==k)>0:
            distr += [multivariate_normal(np.mean(X[clusters==k,:], axis=0), np.cov(X[clusters==k,:].T))]
        else:
            distr += [multivariate_normal(np.mean(X, axis=0), np.cov(X.T))]
    return distr

In [ ]:
# Print
distr0 = compute_distributions(X, clusters0)
print("Mean of the first distribution: \n", distr0[0].mean)
print("\nVariance of the first distribution: \n", distr0[0].cov)

Traçando as distribuições
Vamos adicionar as distribuições ao gráfico.

In [ ]:
# Plot
plot_assignment_gmm(X, clusters0, distr0, i=0, logL=0.0)

Probabilidade
A principal diferença em relação ao K-means é que agora podemos calcular a probabilidade de cada observação pertencer a cada cluster. Essa é a probabilidade de cada observação ter sido gerada por uma das duas distribuições normais bivariadas. Essas probabilidades são chamadas de verossimilhanças .

In [ ]:
# Print first 5 likelihoods
pdfs0 = np.stack([d.pdf(X) for d in distr0], axis=1)
pdfs0[:5]

In [ ]:
Etapa 3: atribuir dados aos clusters
Agora podemos atribuir os dados aos clusters, via máxima verossimilhança.

In [ ]:
# Assign X to clusters
def assign_to_cluster_gmm(X, distr):
    pdfs = np.stack([d.pdf(X) for d in distr], axis=1)
    clusters = np.argmax(pdfs, axis=1)
    log_likelihood = 0
    for k, pdf in enumerate(pdfs):
        log_likelihood += np.log(pdf[clusters[k]])
    return clusters, log_likelihood

In [ ]:
# Get cluster assignment
clusters1, logL1 = assign_to_cluster_gmm(X, distr0)

Plotando dados atribuídos

In [ ]:
# Compute new distributions
distr1 = compute_distributions(X, clusters1)

In [ ]:
# Plot
plot_assignment_gmm(X, clusters1, distr1, 1, logL1);

Expectativa - Maximização
Os dois passos que acabamos de ver fazem parte de uma família mais ampla de algoritmos para maximizar probabilidades, chamados algoritmos de maximização de expectativas .

Na etapa de expectativa, calculamos a expectativa dos parâmetros, dada a atribuição atual do cluster.

Na etapa de maximização, atribuímos observações ao cluster que maximizaram a probabilidade da observação única.

O procedimento alternativo, mais intensivo em termos computacionais, teria sido especificar uma função de verossimilhança global e encontrar os parâmetros de média e variância das duas distribuições normais que maximizaram essas verossimilhanças.

Algoritmo completo
Agora podemos implementar o algoritmo completo.

In [ ]:
def gmm_manual(X, K):

    # Init
    i = 0
    logL0 = 1e4
    logL1 = 1e5
    clusters = np.random.randint(K,size=(np.size(X,0)))

    # Iterate until convergence
    while np.abs(logL0-logL1) > 1e-10:
        logL1 = logL0
        distr = compute_distributions(X, clusters)
        clusters, logL0 = assign_to_cluster_gmm(X, distr)
        plot_assignment_gmm(X, clusters, distr, i, logL0)
        i+=1

Plotando o agrupamento k-means

In [ ]:
# Test
gmm_manual(X, K)

Neste caso, o GMM faz um trabalho muito ruim na identificação dos clusters originais.

Clusters sobrepostos
Vamos agora tentar com um conjunto de dados diferente, onde os dados são extraídos de duas distribuições gaussianas bivariadas sobrepostas, formando uma cruz.

In [ ]:
# Simulate data
X = np.random.randn(50,2)
X[0:25, :] = np.random.multivariate_normal([0,0], [[50,0],[0,1]], size=25)
X[25:, :] = np.random.multivariate_normal([0,0], [[1,0],[0,50]], size=25)

In [ ]:
make_new_figure_1(X)

GMM com distribuições sobrepostas

In [ ]:
### GMM
gmm_manual(X, K)

Como podemos ver, o GMM é capaz de recuperar corretamente os clusters originais.

K-means com distribuições sobrepostas

In [ ]:
### K-means
kmeans_manual(X, K)

K-means gera clusters completamente diferentes.